In [1]:
from pandora_bayesopt.test_functions.robot_pushing.robot_pushing import robot_pushing_14d

pygame 2.1.0 (SDL 2.0.16, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
from torch import Tensor
import numpy as np
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.ei_puc import ExpectedImprovementWithCost
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.acquisition.multi_step_ei import MultiStepLookaheadEI
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer

In [3]:
# use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
seed = 42
torch.manual_seed(seed)

In [5]:
target_location = 10.0 * torch.rand([2]) - 5.0
target_location2 = 10.0 * torch.rand([2]) - 5.0

In [6]:
target_location, target_location2

(tensor([-4.4185, -4.3709]), tensor([-3.7641, -4.4742]))

In [12]:
def unnorm_X(X: torch.Tensor) -> torch.Tensor:
    X_unnorm = X.clone()
    # Check if the tensor is higher than 3-dimensional
    if X.dim() > 3:
        # Assuming the extra unwanted dimension is at position 1 (the second position)
        X_unnorm = X_unnorm.view(-1, X.size(-1))  # Remove the singleton dimension
    # Check the dimensionality of X and adjust accordingly
    if X.dim() == 3:
        # Remove the singleton dimension assuming it's the second one
        X_unnorm = X_unnorm.squeeze(1)
    elif X.dim() == 1:
        # If 1-dimensional, add a dimension to make it 2D (e.g., for batch size of 1)
        X_unnorm = X_unnorm.unsqueeze(0)
    X_unnorm[:, :2] = 10.0 * X_unnorm[:, :2] - 5.0
    X_unnorm[:, 2:4] = 5 * X_unnorm[:, 2:4]
    X_unnorm[:, 4] = 29.0 * X_unnorm[:, 4] + 1.0
    X_unnorm[:, 5] = 2 * np.pi * X_unnorm[:, 5]
    X_unnorm[:, 6:8] = 10.0 * X_unnorm[:, 6:8] - 5.0
    X_unnorm[:, 8:10] = 5 * X_unnorm[:, 8:10]
    X_unnorm[:, 10] = 29 * X_unnorm[:, 10] + 1.0
    X_unnorm[:, 11] = 2 * np.pi * X_unnorm[:, 11]
    X_unnorm[:, 12:] = 2 * np.pi * X_unnorm[:, 12:]

    return X_unnorm

def objective_function(X: torch.Tensor) -> torch.Tensor:
    X_unnorm = unnorm_X(X)
    objective_X = []
    for x in X_unnorm:
        np.random.seed(0)
        object_location, object_location2, robot_location, robot_location2 = torch.tensor(robot_pushing_14d(x[0].item(), x[1].item(), x[2].item(), x[3].item(), x[4].item(), x[5].item(), x[6].item(), x[7].item(), x[8].item(), x[9].item(), x[10].item(), x[11].item(), x[12].item(), x[13].item()))
        objective_X.append(-torch.dist(target_location, object_location)-torch.dist(target_location2, object_location2))
    np.random.seed()  # Reset the seed
    
    return torch.tensor(objective_X)

In [8]:
seed = 1
torch.manual_seed(seed)

In [9]:
dim = 14
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

In [10]:
init_x

tensor([[0.2677, 0.3618, 0.7123, 0.0310, 0.5429, 0.3926, 0.8598, 0.8567, 0.9390,
         0.6023, 0.6899, 0.1173, 0.8344, 0.0827],
        [0.7407, 0.6302, 0.7171, 0.5266, 0.4053, 0.0307, 0.8213, 0.0118, 0.0054,
         0.9607, 0.5815, 0.1177, 0.4939, 0.2619],
        [0.6397, 0.9895, 0.6932, 0.4126, 0.3130, 0.9818, 0.7207, 0.5350, 0.6260,
         0.2308, 0.9248, 0.2234, 0.9211, 0.9408],
        [0.5096, 0.9172, 0.2922, 0.5778, 0.6194, 0.2053, 0.3703, 0.5456, 0.2116,
         0.8923, 0.0013, 0.2025, 0.9183, 0.3255],
        [0.7837, 0.5901, 0.3254, 0.4888, 0.8794, 0.1323, 0.6752, 0.5275, 0.6865,
         0.1164, 0.8389, 0.5008, 0.2496, 0.5004],
        [0.1528, 0.5619, 0.7670, 0.6909, 0.6614, 0.5812, 0.3681, 0.1230, 0.0721,
         0.7912, 0.8940, 0.5347, 0.5987, 0.3354],
        [0.2429, 0.7513, 0.4091, 0.7259, 0.5667, 0.3851, 0.9339, 0.1507, 0.7085,
         0.3987, 0.9743, 0.0327, 0.7866, 0.5466],
        [0.1582, 0.0296, 0.1630, 0.7866, 0.1217, 0.8445, 0.5881, 0.9087, 0.3270,
  

In [13]:
objective_function(init_x)

tensor([-14.7659, -12.2576, -12.2576, -12.2576, -12.2576, -13.4437, -12.2576,
        -12.2576, -12.2576, -12.2576, -12.2576, -12.2576, -14.0308, -12.2576,
        -12.3664, -12.6729, -12.3685, -12.2576, -12.8875, -12.2576, -13.5099,
        -12.2576, -12.7769, -12.3122, -12.2576, -12.2576, -12.2576, -11.9910,
        -11.9794, -12.2576])

In [14]:
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from botorch.utils.sampling import draw_sobol_samples
from botorch.acquisition.predictive_entropy_search import qPredictiveEntropySearch

In [15]:
maximize = True
input_standardize = True
num_iterations = 2
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x, 
        input_standardize=input_standardize
    )
Optimizer.run(
            num_iterations=num_iterations,
            acquisition_function_class = qPredictiveEntropySearch
        )

Iteration 0, New point: [0.74732677 0.589306   0.31848285 0.59459996 0.53361456 0.60007322
 0.81595371 0.41642292 0.64887248 0.50181698 0.78339512 0.13239732
 0.7147757  0.90248912], New value: -12.257621159724565
Best observed value: -11.979440263560832
Current acquisition value: 0.19802843534595094
Cumulative cost: 1.0
Running time: 129.607935

Iteration 1, New point: [0.63867621 0.64245318 0.18919789 0.67612231 0.38800106 0.76245487
 0.62490155 0.63415543 0.54403358 0.75370564 0.46894034 0.53042456
 0.02266885 0.33417419], New value: -12.314867938948597
Best observed value: -11.979440263560832
Current acquisition value: 0.19762863015150495
Cumulative cost: 2.0
Running time: 136.79211400000003



In [13]:
maximize = True
input_standardize = True
num_iterations = 2
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x, 
        input_standardize=input_standardize
    )
Optimizer.run(
            num_iterations=num_iterations,
            acquisition_function_class = MultiStepLookaheadEI
        )

Iteration 0, New point: [0.46998962 0.51364129 0.23806391 0.66094288 0.40406133 0.49301577
 0.83252579 0.41030259 0.61191021 0.73049367 0.86253172 0.07536587
 0.88080394 0.85809839], New value: -12.714719352648768
Best observed value: -11.979440263560832
Current acquisition value: tensor(0.4643)
Cumulative cost: tensor(1.)
Running time: 777.3063669999999



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 3 and Y with dimension 4).
  warnings.warn(
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTensorDimensionWarning: Non-strict enforcement of botorch tensor conventions. The following error would have been raised with strict enforcement: Expected X and Y to have the same number of dimensions (got X with dimension 4 and Y with dimension 5).
  warnings.warn(
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/gpytorch.py:99: BotorchTens

Iteration 1, New point: [0.46458694 0.25404045 0.17119677 0.6043153  0.26967387 0.53810771
 0.70317251 0.62482248 0.5450931  1.         0.76217294 0.11723842
 0.99513089 0.90768712], New value: -12.962865576872105
Best observed value: -11.979440263560832
Current acquisition value: tensor(0.5008)
Cumulative cost: tensor(2.)
Running time: 1043.857674



In [11]:
maximize = True
input_standardize = True
num_iterations = 2
Optimizer = BayesianOptimizer(
        objective=objective_function, 
        dim=dim, 
        maximize=maximize, 
        initial_points=init_x, 
        input_standardize=input_standardize
    )
Optimizer.run(
            num_iterations=num_iterations,
            acquisition_function_class = GittinsIndex,
            step_divide = True,
            alpha=2
        )

Iteration 0, New point: [0.63179582 0.38636396 0.33572468 0.59879379 0.47067196 0.61691711
 0.76765401 0.45453309 0.64104743 0.70200081 0.80235899 0.15181769
 0.91285118 0.86440273], New value: -12.071019539287862
Best observed value: -11.979440263560832
Current acquisition value: -11.922940127479361
Cumulative cost: 1.0
Gittins lmbda: 0.1
Running time: 19.823867999999997

Iteration 1, New point: [0.44111386 0.53332271 0.22080603 0.66361186 0.39190687 0.45327023
 0.84289532 0.39394119 0.61250603 0.74397337 0.88299684 0.06689219
 0.8794775  0.85707015], New value: -13.042746450735962
Best observed value: -11.979440263560832
Current acquisition value: -11.928444233224031
Cumulative cost: 2.0
Gittins lmbda: 0.1
Running time: 18.535795999999998

